# Defensive programming

**Note**: This lesson draws heavily from, and in some parts quotes directly,
the [Python Testing](http://katyhuff.github.io/python-testing/) lesson
developed by Kathryn Huff. 

Untested code is broken code. Doing science with untested code is akin to using
an experimental device that is uncalibrated, which is generally a bad idea.
The best way to write code that works and keeps on working is to assume it's
broken, and to build yourself some alarms for when its behavior is outside of
what is expected. 

This mindset is often called **defensive programming**.

In this lesson, we will learn about various flavors of testing code, including:

1. [Assertions](http://katyhuff.github.io/python-testing/02-assertions.html)
2. [Exceptions](http://katyhuff.github.io/python-testing/03-exceptions.html)
3. [Unit Tests](http://katyhuff.github.io/python-testing/04-units.html)

and we will write some of our own, too.

#### Additional resources

* [Python Testing](http://katyhuff.github.io/python-testing/) by Kathryn Huff
* _[Effective Computation in Physics, Chapter 18](http://physics.codes/)_, A. Scopatz and K. Huff. O'Reilly Media. (2015)


## Using assertions

Learning objectives:
* Assertions are one line tests embedded in code.
* Assertions can halt execution if something unexpected happens.
* Assertions are the building blocks of tests.

The ``assert`` Python keyword tests the truth value of what follows, and if what follows evaluates to ``False``, then it raises an ``AssertionError`` (a type of ``Exception``, which we'll get to):

In [2]:
assert True == True

In [4]:
assert True == False, "True is not False"

AssertionError: True is not False

We can follow up an assertion statement with a string giving what should be printed when the assertion rings false. We'll see how this is useful below.

### Assertions as input enforcement

A common use of assertions is to check that the inputs of a function 
meet the expectations of that function; that is, that they are valid
given the assumptions that function needs to make about them. If we
write a simple function that gets the mean of a list of values:

In [2]:
def mean(num_list):
    return sum(num_list)/len(num_list)

In [3]:
mean([4, 2, 3])

3.0

Feeding it an empty list gives:

In [4]:
mean([])

ZeroDivisionError: division by zero

Let's add some assertions. We'll also add in a docstring to make clear
what we want this function to take, and what we want it to return.

In [6]:
def mean(num_list):
    """Return the mean of a list of numbers.
    
    Parameters
    ----------
    num_list : list
        List of values to get arithmetic mean for.
        
    Returns
    -------
    float
        Arithmetic mean.
    """
    assert len(num_list) > 0, "Cannot take an empty list"
    assert all([isinstance(i, (float, int)) for i in num_list]), "List must only have numbers"
    
    return sum(num_list)/len(num_list)

Now we have two assertions that check:
1. the list given is not empty.
2. all elements in the list are either floating point numbers or integers.

So now when we do:

In [7]:
mean([])

AssertionError: Cannot take an empty list

we get back an error message that's more meaningful to us than "cannot divide by zero". We can change how we're using the function appropriately, and didn't have to dig into the implementation to understand what went wrong.

Also, if we give it:

In [8]:
mean([42, "a word"])

AssertionError: List must only have numbers

we see that our second assertion catches cases where the list has non-numbers, and complains clearly why it fails.

## Using exceptions as flexible, catchable assertions

Assertions are useful for input-checking, but in production code 
it's generally better to explicitly use **exceptions**. 
An ``AssertionError`` is one type of exception, but we can use
others to greater effect.

Let's change our ``mean`` function to raise a ``ValueError`` instead
of an ``AssertionError`` when we give an empty list:

In [10]:
def mean(num_list):
    if len(num_list) == 0:
        raise ValueError("The arithmetic mean of no elements makes no sense")
    return sum(num_list)/len(num_list)

In [11]:
mean([1, 2, 3])

2.0

In [12]:
mean([])

ValueError: The arithmetic mean of no elements makes no sense

Raising a ``ValueError`` more [clearly defines the type of error
indicated](https://docs.python.org/3/library/exceptions.html#ValueError): we gave a list, but it was empty. We'll see how using different types of exceptions allows us to write more flexible code below.

### Catching exceptions

Let's rewrite our ``mean`` function yet again, only this time we'll put
the meat of the function--the actual calculation--inside a ``try-except`` block. 

In [13]:
def mean(num_list):
    try:
        return sum(num_list)/len(num_list)
    except ZeroDivisionError:
        return 0

Instead of raising an exception giving ``mean`` an empty list, we could
catch the ``ZeroDivisionError`` raised by the calculation and simply
return ``0``, which sounds sensible. It's up to us how our function
behaves, but choosing sensible behavior is a good idea.

In [14]:
mean([])

0

Can we do something similar for the case where the list has non-number
elements? Yes.

In [24]:
def mean(num_list):
    try:
        return sum(num_list)/len(num_list)
    except ZeroDivisionError:
        return 0
    except TypeError:
        raise TypeError("Cannot get mean for non-number elements")

In [25]:
mean([1, "nothing"])

TypeError: Cannot get mean for non-number elements

In this case we caught the ``TypeError`` that results from getting the
sum of a list with non-number elements, then we raised another
``TypeError`` (since this is a good choice of exception for this
issue) with a more descriptive message that tells us what is wrong.

## Defining expected behavior with unit tests

Assertions and exceptions give mechanisms for checking that functions
are working as expected at runtime, with the inputs given to them
at runtime. But these don't tell us how the function will behave
for inputs that it *might* get elsewhere at other times. How can we
ensure that our function behaves as we expect for different assortments
of input?

We can write **unit tests**.

Let's place the last version of our ``mean`` function into a module
called ``mean.py``; open your favorite text editor and make your ``mean.py`` look like this:

In [28]:
%cat mean.py

def mean(num_list):
    try:
        return sum(num_list)/len(num_list)
    except ZeroDivisionError:
        return 0
    except TypeError:
        raise TypeError("Cannot get mean for non-number elements")


We can import this mean function directly, and use it as before:

In [29]:
from mean import mean

In [30]:
mean([5])

5.0

In [31]:
mean([])

0

Now make a file in the same directory called ``test_mean.py`` in your
favorite text editor, and put a single function called ``test_ints``
inside. Don't forget to import your ``mean`` function at the top
of this new module:

In [33]:
%cat test_mean.py

from mean import mean

def test_ints():
    num_list = [1, 2, 3, 4, 5]
    obs = mean(num_list)

    assert obs == 3


Congratulations! You've written your first unit test. This simple test
function takes the mean of a known list of numbers, and then at the
end asserts that the result is what we know whould be the answer. We
can run this using ``py.test`` in the shell from the same directory:

In [36]:
%%bash
py.test

============================= test session starts ==============================
platform linux -- Python 3.5.1, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /home/alter/Library/becksteinlab/ComputationalPhysics494/PHY494-resources/14_testing, inifile: 
collected 1 items

test_mean.py .

=========================== 1 passed in 0.01 seconds ===========================


[py.test](http://pytest.org/) is one widely-used and 
actively-developed testing framework. It can do way more than we are
going to use here, but it makes complex sets of tests much easier to
build than otherwise.

If we add more tests to our test suite, such that we now have:

In [37]:
%cat test_mean.py

from mean import mean

import pytest

def test_ints():
    num_list = [1, 2, 3, 4, 5]
    obs = mean(num_list)

    assert obs == 3

def test_not_numbers():
    values = [2, "lolcats"]
    with pytest.raises(TypeError):
        out = mean(values)

def test_zero():
    num_list = [0, 2, 4, 6]
    assert mean(num_list) == 3

def test_empty():
    assert mean([]) == 0

def test_single_int():
    with pytest.raises(TypeError):
        mean(1)



Note the special **context manager** ``pytest.raises`` used to assert that the statement that follows raises a particular exception. These are useful for making sure our function gives the expected response for
gnarly inputs.

We see that ``py.test`` finds these as well:

In [38]:
%%bash
py.test

============================= test session starts ==============================
platform linux -- Python 3.5.1, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /home/alter/Library/becksteinlab/ComputationalPhysics494/PHY494-resources/14_testing, inifile: 
collected 5 items

test_mean.py .....

=========================== 5 passed in 0.02 seconds ===========================


For each test that passes, a `.` is printed. If a test failed, we'd get an `F` in that place instead, and ``py.test`` would tell us where the
failure occurred. Let's change our function so that it returns ``None``
instead of ``0`` for an empty list to see if this affects our tests:

In [39]:
%cat mean.py

def mean(num_list):
    try:
        return sum(num_list)/len(num_list)
    except ZeroDivisionError:
        return None
    except TypeError:
        raise TypeError("Cannot get mean for non-number elements")


In [40]:
%%bash
py.test

============================= test session starts ==============================
platform linux -- Python 3.5.1, pytest-2.9.1, py-1.4.31, pluggy-0.3.1
rootdir: /home/alter/Library/becksteinlab/ComputationalPhysics494/PHY494-resources/14_testing, inifile: 
collected 5 items

test_mean.py ...F.

=================================== FAILURES ===================================
__________________________________ test_empty __________________________________

    def test_empty():
>       assert mean([]) == 0
E       assert None == 0
E        +  where None = mean([])

test_mean.py:21: AssertionError
====================== 1 failed, 4 passed in 0.03 seconds ======================


This small change was caught by one of our tests, and we see exactly
where. This is where unit tests become immensely useful: 
if ``mean`` was part of a larger codebase and we decided to make a tiny
change to it, we see immediately that this change affects the behavior
expected of it by our tests. We can the decide if we **want** the new
behavior (so we'd change the tests) or if the new behavior was a
mistake (so we'd fix the function). 

Without tests, it is very hard to ensure a large amount of code
continues behaving as we expect while we (and maybe others) keep
working at it. The more tests we have, the more well-defined the
expected behavior of the codebase, and the more time you will save
scratching your head later.